# Choose scenario (users' actions needed)

In [ ]:
region = "CNA" # either "CNA" or "EAS", choose the region
RCP = "RCP45" # either "RCP45" or "RCP85", choose the RCP level
model = "Full" # either ="Full" or "SSB", choose the model, "Full" is our model and "SSB" is the model proposed by Sansom, P. G., D. B. Stephenson, and T. J. Bracegirdle (2017). On constraining projections of future climate using observations and simulations from multiple climate models. 
nSave = 500 # save results after multiplications of this number of iterations is reached
nThin = 5 # the number of thinning, i.e., only one iteration is saved for every nThin iterations
nChain = 20000 # the number of saved MCMC iterations 
nBurn = 30000 # the number of iterations for burn-in

# Check user variables are correctly set

## Capitalize string variables to avoid mistakes

In [ ]:
region = region.upper()
RCP = RCP.upper()
model =  model.upper()

## Make sure nSave, nThin, nChain, nBurn are integers

In [ ]:
nSave = int(nSave)
nThin = int(nThin)
nChain = int(nChain)
nBurn = int(nBurn)

## Check variable values

In [ ]:
if (region != "CNA") and (model != "EAS"):
    print('\x1b[1;31m'+'Error: the variable "region" must be either "CNA" or "EAS"!'+'\x1b[0m')
if (model != "FULL") and (model != "SSB"):
    print('\x1b[1;31m'+'Error: the variable "model" must be either "FULL" or "SSB"!'+'\x1b[0m')
if (RCP != "RCP45") and (model != "RCP85"):
    print('\x1b[1;31m'+'Error: the variable "RCP" must be either "RCP45" or "RCP85"!'+'\x1b[0m')
if nSave < 1:
    print('\x1b[1;31m'+'Error: the variable "nSave" must be greater than 0!'+'\x1b[0m')
if nThin < 1:
    print('\x1b[1;31m'+'Error: the variable "nThin" must be greater than 0!'+'\x1b[0m')
if nChain < 1:
    print('\x1b[1;31m'+'Error: the variable "nChain" must be greater than 0!'+'\x1b[0m')
if nBurn < 1:
    print('\x1b[1;31m'+'Error: the variable "nBurn" must be greater than 0!'+'\x1b[0m')

# File names

In [ ]:
dataName="data/{}-{}".format(region,RCP)
resultName="results/{}-{}-{}".format(region,RCP,model)

# Import modules

In [ ]:
from scipy import linalg as spl
import numpy as np
import sys
import shelve

# Import user modules

In [ ]:
sys.path.insert(0,"../src")

import state
import result
from tools import *
from estimate_Y import *
from estimate_X import *
from estimate_X_withoutSpatial import *

from estimate_mu import *
from estimate_tauW import *

from estimate_phi import *

from estimate_phi_m import *
from estimate_phi_m_withoutSpatial import *

from estimate_gamma_m import *

from estimate_phi_a import *
from estimate_nu import *

from estimate_V import *
from estimate_tau import *
from estimate_beta import *
from estimate_gamma import *

# Read Data

## Load CMIP data

In [ ]:
## XHmr: dim: M * RHm[m] * n
## XFmr: dim: M * RFm[m] * n
## XHm: dim: M * n
## XFm: dim: M * n
## W: dim: N * n
## dist: dim: n * n
## V: dim: M * M

read_application_data(dataName);

state.RHm = state.RHm.astype("int32")
state.RFm = state.RFm.astype("int32")

## Load coordinates

In [ ]:
db = dbm.dumb.open('data/Coordinates','r')
my_shelf = shelve.Shelf(db)

if region == "CNA":
    lon = my_shelf['cna_lon']
    lat = my_shelf['cna_lat']
else:
    lon = my_shelf['eas_lon']
    lat = my_shelf['eas_lat']
my_shelf.close()

## Scale distances so that the length of longest dimension is 1

In [ ]:
scale = max(abs(lon.max()-lon.min()),abs(lon.max()-lon.min()))
state.dist /= scale

# Initial parameter assignments

In [ ]:
state.nChain = nChain
state.nBurn = nBurn
assign_parameters();

if model == "SSB":
    state.covMatH = np.identity(state.n)
    state.covMatF = np.identity(state.n)

    state.invCovMatH = np.identity(state.n)
    state.invCovMatF = np.identity(state.n)
    
    result.gammaHm = np.nan
    result.gammaFm = np.nan

    result.gammaH = np.nan
    result.gammaF = np.nan

    result.V = np.nan
    result.V_one = np.nan
    result.V_mean = np.nan

# Run MCMC

## Burn-in

In [ ]:
for state.iteration in range(nBurn):
    estimate_Y()
    if model == "FULL":
        estimate_X()
    else:
        estimate_X_withoutSpatial()
    
    estimate_mu()
    estimate_tauW()
    estimate_phi()
    
    if model == "FULL":
        estimate_phi_m()
        estimate_gamma_m(0.02,0.012)
    else:
        estimate_phi_m_withoutSpatial()
        
    estimate_phi_a()
    estimate_nu()
    if model == "FULL":
        estimate_V()
    estimate_tau()
    estimate_beta()
    if model == "Full":
        estimate_gamma(0.08,0.08)

## Effective runs

In [ ]:
for state.iteration in range(nChain):
    for thin in range(nThin):
        estimate_Y()
        if model == "FULL":
            estimate_X()
        else:
            estimate_X_withoutSpatial()

        estimate_mu()
        estimate_tauW()
        estimate_phi()

        if model == "FULL":
            estimate_phi_m()
            estimate_gamma_m(0.02,0.012)
        else:
            estimate_phi_m_withoutSpatial()

        estimate_phi_a()
        estimate_nu()
        if model == "FULL":
            estimate_V()
        estimate_tau()
        estimate_beta()
        if model == "FULL":
            estimate_gamma(0.08,0.08)
        
    result.YH[state.iteration] = state.YH
    result.YF[state.iteration] = state.YF
    result.YHa_one[state.iteration] = state.YHa[0]
    result.YFa_one[state.iteration] = state.YFa[0]  
    result.YHa_mean += state.YHa
    result.YFa_mean += state.YFa

    result.XHm_one[state.iteration] = state.XHm[0,0]
    result.XFm_one[state.iteration] = state.XFm[0,0]
    result.XHm_mean += np.mean(state.XHm,axis=0)
    result.XFm_mean += np.mean(state.XFm,axis=0)

    result.muH[state.iteration] = state.muH
    result.muF[state.iteration] = state.muF
    
    result.tauW[state.iteration] = state.tauW

    result.phiH[state.iteration] = state.phiH
    result.phiF[state.iteration] = state.phiF
     
    result.phiHm[state.iteration] = state.phiHm
    result.phiFm[state.iteration] = state.phiFm
    
    result.phiHa[state.iteration] = state.phiHa
    result.phiFa[state.iteration] = state.phiFa
    
    result.nuH[state.iteration] = state.nuH
    result.nuF[state.iteration] = state.nuF

    result.tauH[state.iteration] = state.tauH
    result.tauF[state.iteration] = state.tauF
    
    result.beta[state.iteration] = state.beta
    
    if model == "FULL":
        result.gammaHm[state.iteration] = state.gammaHm * scale
        result.gammaFm[state.iteration] = state.gammaFm * scale

        result.gammaH[state.iteration] = state.gammaH * scale
        result.gammaF[state.iteration] = state.gammaF * scale
        
        result.V[state.iteration] = state.V
        result.V_one[state.iteration] = state.V[19,20]
        result.V_mean += state.V

    # Save results constantly
    if state.iteration%nSave == 0:
        save_data(resultName)
        print("Iteration {} completes.".format(state.iteration))

# Save final results

In [ ]:
state.iteration += 1
if state.iteration < state.nChain: state.nChain = iteration

result.XHm_mean /= state.nChain
result.XFm_mean /= state.nChain

result.YHa_mean /= state.nChain
result.YFa_mean /= state.nChain

result.V_mean /= state.nChain

In [ ]:
save_data(resultName);